In [1]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras as extras
import pandas as pd

In [2]:
param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "taxis",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

In [3]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connection succesful')
    
    return conn

conn = connect(param_dic)
cursor = conn. cursor()

Connecting to the PostgreSQL database...
Connection succesful


In [4]:
def time_lapse(seconds):
    seconds_in_day = 60 * 60 * 24
    seconds_in_hour = 60 * 60
    seconds_in_minute = 60
    
    days = seconds // seconds_in_day
    hours = (seconds - (days * seconds_in_day)) // seconds_in_hour
    minutes = (seconds - (days * seconds_in_day) - (hours * seconds_in_hour)) // seconds_in_minute
    
    return f"{days} days, {hours} hours, {minutes} minutes"

# Parte 1 - SQL

1. 
    a) ¿Puedes describirnos el procedimiento que has seguido para cargar los tres ficheros?

- Mediante Requests solicito el html vía http del link proporcionado.
- Con BeautifoulSoup filtro por los div por año mediante el tag id="faq{year}". 
- De estos divs seleceiono los tag 'ul' (que contienen los links csv href) para los meses indicados

1.    
    b) ¿Puedes compartirnos la definición de la tabla que estás usando para contestar las
preguntas planteadas?

2. Una vez cargados los tres ficheros en tu base de datos, ¿cuántos registros tiene la
tabla? Compártenos la SQL que has usado.

In [5]:
registros = '''SELECT COUNT(vendorid) AS registros_totales
FROM yellowtaxis;
'''
conn.rollback()
cursor.execute(registros)
result = cursor.fetchall()[0]
print(f"La tabla yellowtaxis de la database {conn.info.dbname} contiene {result[0]} registros")

La tabla yellowtaxis de la database taxis contiene 15711654 registros


3. Para cada uno de los tres meses con datos ¿podrías decirnos cuál es el trayecto más
corto, más largo y el trayecto medio?, en distancia y en tiempo. Compártenos la
sentencia SQL que has usado.


In [6]:
def execute(month: int, trip_metric, trip_measure):
    
    query = None
    if trip_metric == 'longest':
        if trip_measure == 'time':
            longest_trip_time = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime, trip_duration_seconds, trip_duration
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_duration_seconds DESC
                LIMIT 1;
            '''
            query = longest_trip_time
        elif trip_measure == 'distance':
            longest_trip_distance = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime,
                trip_duration_seconds, trip_duration, trip_distance
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_distance DESC
                LIMIT 1;
            '''
            query = longest_trip_distance
            
    elif trip_metric == 'shortest':
        if trip_measure == 'time':
            shortest_trip_time = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_duration_seconds ASC,
                trip_distance ASC
                LIMIT 1;

            '''
            query = shortest_trip_time
            
        elif trip_measure == 'distance':
            shortest_trip_distance = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_distance ASC,
                trip_duration_seconds ASC
                LIMIT 1;

            '''
            query = shortest_trip_distance
    
    elif trip_metric == 'average':
        if trip_measure == 'time':
            avg_trip_time = f'''
                SELECT AVG(trip_duration_seconds) AS trip_duration_avg
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query = avg_trip_time
            
        elif trip_measure == 'distance':
            avg_trip_distance = f'''
                SELECT AVG(trip_distance) AS trip_distance_avg
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query =  avg_trip_distance
    
    cursor.execute(query)
    result = cursor.fetchall()[0]
    
    return result

# January

In [7]:
# Longest time
jan_lg_t = execute(1, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from January 2020 took: {jan_lg_t[-1]}")
print()

# Longest distance

jan_lg_d = execute(1, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from January 2020 was: {jan_lg_d[-1]} miles")
print()


Longest yellow taxi trip time from January 2020 took: 5 days, 22 hours, 5 minutes


Longest yellow taxi trip distance from January 2020 was: 210240.07 miles



In [8]:
# Shortest time
jan_st_t = execute(1, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from January 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
jan_st_d = execute(1, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from January 2020 was: {jan_st_d[7]} miles")
print()


Shortest yellow taxi trip time from January 2020 took: 1 seconds


Shortest yellow taxi trip distance from January 2020 was: 0.01 miles



In [9]:
# Average
jan_avg_t =  execute(1, 'average', 'time')
print()
print(f"Average yellow taxi trip time from January 2020 was: {time_lapse(jan_avg_t[0])}")
print()

# Average distance
jan_avg_d = execute(1, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from January 2020 was: {jan_avg_d[0]} miles")
print()


Average yellow taxi trip time from January 2020 was: 0 days, 0 hours, 15 minutes


Average yellow taxi trip distance from January 2020 was: 2.9656103565241363 miles



# February 

In [10]:
feb_lg_t = execute(2, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from February 2020 took: {feb_lg_t[-1]}")
print()

# Longest distance

feb_lg_d = execute(2, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from February 2020 was: {feb_lg_d[-1]} miles")
print()


Longest yellow taxi trip time from February 2020 took: 2 days, 4 hours, 27 minutes


Longest yellow taxi trip distance from February 2020 was: 57051.09 miles



In [11]:
# Shortest time
feb_st_t = execute(2, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {feb_st_t[4]} seconds")
print()

# Shortest distance
feb_st_d = execute(2, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from February 2020 was: {feb_st_d[7]} miles")
print()


Shortest yellow taxi trip time from February 2020 took: 1 seconds


Shortest yellow taxi trip distance from February 2020 was: 0.01 miles



In [12]:
# Average
feb_avg_t =  execute(2, 'average', 'time')
print()
print(f"Average yellow taxi trip time from February 2020 was: {time_lapse(feb_avg_t[0])}")
print()

# Average distance
feb_avg_d = execute(2, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from February 2020 was: {feb_avg_d[0]} miles")
print()


Average yellow taxi trip time from February 2020 was: 0 days, 0 hours, 16 minutes


Average yellow taxi trip distance from February 2020 was: 2.8923296836424925 miles



# March

In [13]:
mar_lg_t = execute(3, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from March 2020 took: {mar_lg_t[-1]}")
print()

# Longest distance
mar_lg_d = execute(3, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from March 2020 was: {mar_lg_d[-1]} miles")
print()


Longest yellow taxi trip time from March 2020 took: 1 days, 6 hours, 14 minutes


Longest yellow taxi trip distance from March 2020 was: 269803.73 miles



In [14]:
# Shortest time
mar_st_t = execute(3, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
mar_st_d = execute(3, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from March 2020 was: {mar_st_d[7]} miles")
print()


Shortest yellow taxi trip time from February 2020 took: 1 seconds


Shortest yellow taxi trip distance from March 2020 was: 0.01 miles



In [15]:
# Average
mar_avg_t =  execute(3, 'average', 'time')
print()
print(f"Average yellow taxi trip time from March 2020 was: {time_lapse(mar_avg_t[0])}")
print()

# Average distance
mar_avg_d = execute(3, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from March 2020 was: {mar_avg_d[0]} miles")
print()


Average yellow taxi trip time from March 2020 was: 0 days, 0 hours, 15 minutes


Average yellow taxi trip distance from March 2020 was: 3.1496015501265515 miles



4. Para cada uno de los tres meses con datos ¿podrías decirnos cuánta es la variación
porcentual en el número de servicios registrados con respecto al mes anterior? Por
ejemplo, en esta pregunta nos gustaría obtener un resultado así. Compártenos la
sentencia SQL que has usado

# Parte 2 - Django

# Parte 3 - Reporting

- Empleando Python 3 generar un fichero Excel que contenga tres pestañas.
- Para este ejercicio las transformaciones queremos que se hagan en Pandas y no una
sentencia SQL
- Eplea el método/libreria de tu preferencia para generar el fichero Excel
- La información que queremos que se muestre en cada una de las pestañas es una
agrupación por mes y tipo de dia:
    - MES: Tendrá el formato año-mes, por ejemplo 2020-01
    -Tipo_dia: Tendrá el valor 1 si el viaje se realizó entre el lunes y el viernes
(ambos inclusive) y 2 si se hizo en fin de semana
    - Personas transportadas: El total de personas que han usado el servicio de Taxis
para ese mes y tipo_dia
    - Millas recorridas: El total de millas recorridas para ese mes y tipo_dia
    - Total servicios: El total de servicios que realizados para ese mes y tipo_dia

- Pestaña JFK: Tendrá la agrupación descrita para todos los viajes que tengan un 2 en la
variable RateCodeID
- Pestaña regular: Tendrá la agrupación descrita para todos los viajes que tengan un 1 en
la variable RateCodeID
- Pestaña Others: Tendrá la agrupación descrita para todos los viajes que no tengan ni un
1 ni un 2 en la variable RateCodeID

In [16]:
############################################################################################################

In [17]:
name = "2020_yellow_jan_feb_mar.csv"
jan_feb_mar = pd.read_csv(name)

/home/david/miniconda3/envs/work/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
jan_feb_mar

,VendorID,month,tpep_pickup_datetime,tpep_dropoff_datetime,trip_duration_seconds,trip_duration,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,...,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,288,"0 days, 0 hours, 4 minutes",1.0,1.20,1.0,N,...,239,1.0,6.00,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,1,2020-01-01 00:35:39,2020-01-01 00:43:04,445,"0 days, 0 hours, 7 minutes",1.0,1.20,1.0,N,...,238,1.0,7.00,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,1,2020-01-01 00:47:41,2020-01-01 00:53:52,371,"0 days, 0 hours, 6 minutes",1.0,0.60,1.0,N,...,238,1.0,6.00,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,1,2020-01-01 00:55:23,2020-01-01 01:00:14,291,"0 days, 0 hours, 4 minutes",1.0,0.80,1.0,N,...,151,1.0,5.50,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,1,2020-01-01 00:01:58,2020-01-01 00:04:16,138,"0 days, 0 hours, 2 minutes",1.0,0.00,1.0,N,...,193,2.0,3.50,0.5,0.5,0.00,0.0,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15711649,NaN,3,2020-03-31 23:21:00,2020-03-31 23:33:00,720,"0 days, 0 hours, 12 minutes",NaN,5.43,NaN,NaN,...,87,NaN,30.47,0.0,0.5,0.00,0.0,0.3,33.77,2.5
15711650,NaN,3,2020-03-31 23:57:00,2020-04-01 00:26:00,1740,"0 days, 0 hours, 29 minutes",NaN,13.21,NaN,NaN,...,71,NaN,37.97,0.0,0.5,0.00,0.0,0.3,41.27,2.5
15711651,NaN,3,2020-03-31 23:22:01,2020-03-31 23:43:52,1311,"0 days, 0 hours, 21 minutes",NaN,12.14,NaN,NaN,...,32,NaN,37.10,0.0,0.0,0.00,0.0,0.3,39.90,2.5
15711652,NaN,3,2020-03-31 23:18:53,2020-03-31 23:32:21,808,"0 days, 0 hours, 13 minutes",NaN,7.01,NaN,NaN,...,159,NaN,20.07,0.0,0.0,0.00,0.0,0.3,22.87,2.5


In [19]:
month_col = 'month'
start_date_col = 'tpep_pickup_datetime'
final_date_col = 'tpep_dropoff_datetime'
passenger_col = 'passenger_count'
distance_col = 'trip_distance'
ratecode_col = 'RatecodeID'

df = jan_feb_mar.copy()

df = df[[month_col, start_date_col, final_date_col, passenger_col, distance_col, ratecode_col]]
df = df[(df[start_date_col] > '2020-01-01') & (df[start_date_col] <= '2020-03-31')]

type_object = df.select_dtypes(include='object').columns.to_list()
for elem in type_object:
        df[f"{elem}"] = pd.to_datetime(df[f"{elem}"], yearfirst=True, format="%Y/%m/%d %H:%M:%S")
        
df['day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
df['tipo_dia'] = df['day_of_week'].apply(lambda x: 1 if x in range(0,5) else 2)
df['total_servicios'] = 1
df['mes'] = df['tpep_pickup_datetime'].dt.strftime('%Y-%m')

df.rename(columns={"trip_distance": "millas_recorridas",
                   "passenger_count":"personas_transportadas"}, 
          inplace=True)


In [20]:
df

,month,tpep_pickup_datetime,tpep_dropoff_datetime,personas_transportadas,millas_recorridas,RatecodeID,day_of_week,tipo_dia,total_servicios,mes
0,1,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,2,1,1,2020-01
1,1,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,2,1,1,2020-01
2,1,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,2,1,1,2020-01
3,1,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,2,1,1,2020-01
4,1,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,2,1,1,2020-01
...,...,...,...,...,...,...,...,...,...,...
15711642,3,2020-03-30 07:02:04,2020-03-30 07:40:21,NaN,0.00,NaN,0,1,1,2020-03
15711643,3,2020-03-30 07:02:20,2020-03-30 07:32:15,NaN,10.61,NaN,0,1,1,2020-03
15711644,3,2020-03-30 07:58:01,2020-03-30 08:03:40,NaN,1.00,NaN,0,1,1,2020-03
15711645,3,2020-03-30 07:17:43,2020-03-30 07:25:56,NaN,2.70,NaN,0,1,1,2020-03


In [21]:
table = df.copy()
table = table[['mes', 'tipo_dia', 'personas_transportadas', 'millas_recorridas', 'total_servicios', 'RatecodeID']]
table

,mes,tipo_dia,personas_transportadas,millas_recorridas,total_servicios,RatecodeID
0,2020-01,1,1.0,1.20,1,1.0
1,2020-01,1,1.0,1.20,1,1.0
2,2020-01,1,1.0,0.60,1,1.0
3,2020-01,1,1.0,0.80,1,1.0
4,2020-01,1,1.0,0.00,1,1.0
...,...,...,...,...,...,...
15711642,2020-03,1,NaN,0.00,1,NaN
15711643,2020-03,1,NaN,10.61,1,NaN
15711644,2020-03,1,NaN,1.00,1,NaN
15711645,2020-03,1,NaN,2.70,1,NaN


- Pestaña JFK: Tendrá la agrupación descrita para todos los viajes que tengan un 2 en la
variable RateCodeID

In [36]:
# Agrupamos por las variables indicadas
jfk = table.copy()
jfk = jfk.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
jfk

,mes,tipo_dia,RatecodeID,personas_transportadas,millas_recorridas,total_servicios
0,2020-01,1,1.0,6952623.0,1.139321e+07,4663589
1,2020-01,1,2.0,185080.0,1.899542e+06,117768
2,2020-01,1,3.0,16205.0,1.673550e+05,10366
3,2020-01,1,4.0,5765.0,7.535857e+04,3864
4,2020-01,1,5.0,37176.0,1.203135e+05,26046
5,2020-01,1,6.0,39.0,2.497000e+01,40
6,2020-01,1,99.0,243.0,3.367000e+02,280
7,2020-01,2,1.0,2323876.0,3.650608e+06,1463042
8,2020-01,2,2.0,63469.0,6.354032e+05,39963
9,2020-01,2,3.0,5116.0,4.882800e+04,3109


In [37]:
# Filtramos RaatecodeID = 2
jfk = jfk[jfk['RatecodeID'] == 2]
jfk

,mes,tipo_dia,RatecodeID,personas_transportadas,millas_recorridas,total_servicios
1,2020-01,1,2.0,185080.0,1899541.76,117768
8,2020-01,2,2.0,63469.0,635403.23,39963
15,2020-02,1,2.0,156621.0,1631009.44,100800
22,2020-02,2,2.0,63710.0,633585.90,40030
29,2020-03,1,2.0,63527.0,660936.64,41420
36,2020-03,2,2.0,27093.0,276021.14,17478


In [24]:
jfk.drop(columns=['RatecodeID'], inplace=True)
jfk

/home/david/miniconda3/envs/work/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,mes,tipo_dia,personas_transportadas,millas_recorridas,total_servicios
1,2020-01,1,185080.0,1899541.76,117768
8,2020-01,2,63469.0,635403.23,39963
15,2020-02,1,156621.0,1631009.44,100800
22,2020-02,2,63710.0,633585.90,40030
29,2020-03,1,63527.0,660936.64,41420
36,2020-03,2,27093.0,276021.14,17478


In [25]:
#table.groupby('mes').sum().reset_index()

- Pestaña regular: Tendrá la agrupación descrita para todos los viajes que tengan un 1 en
la variable RateCodeID

In [31]:
regular = table.copy()
regular = regular.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
regular

,mes,tipo_dia,RatecodeID,personas_transportadas,millas_recorridas,total_servicios
0,2020-01,1,1.0,6952623.0,1.139321e+07,4663589
1,2020-01,1,2.0,185080.0,1.899542e+06,117768
2,2020-01,1,3.0,16205.0,1.673550e+05,10366
3,2020-01,1,4.0,5765.0,7.535857e+04,3864
4,2020-01,1,5.0,37176.0,1.203135e+05,26046
5,2020-01,1,6.0,39.0,2.497000e+01,40
6,2020-01,1,99.0,243.0,3.367000e+02,280
7,2020-01,2,1.0,2323876.0,3.650608e+06,1463042
8,2020-01,2,2.0,63469.0,6.354032e+05,39963
9,2020-01,2,3.0,5116.0,4.882800e+04,3109


In [32]:
regular = regular[regular['RatecodeID'] == 1]
regular

,mes,tipo_dia,RatecodeID,personas_transportadas,millas_recorridas,total_servicios
0,2020-01,1,1.0,6952623.0,1.139321e+07,4663589
7,2020-01,2,1.0,2323876.0,3.650608e+06,1463042
14,2020-02,1,1.0,6381098.0,1.044654e+07,4329379
21,2020-02,2,1.0,2725750.0,4.276544e+06,1728079
28,2020-03,1,1.0,3127472.0,5.351717e+06,2166842
35,2020-03,2,1.0,1093646.0,1.862253e+06,710739


In [33]:
regular.drop(columns=['RatecodeID'], inplace=True)
regular

/home/david/miniconda3/envs/work/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,mes,tipo_dia,personas_transportadas,millas_recorridas,total_servicios
0,2020-01,1,6952623.0,1.139321e+07,4663589
7,2020-01,2,2323876.0,3.650608e+06,1463042
14,2020-02,1,6381098.0,1.044654e+07,4329379
21,2020-02,2,2725750.0,4.276544e+06,1728079
28,2020-03,1,3127472.0,5.351717e+06,2166842
35,2020-03,2,1093646.0,1.862253e+06,710739


- Pestaña Others: Tendrá la agrupación descrita para todos los viajes que no tengan ni un
1 ni un 2 en la variable RateCodeID

In [38]:
others = table.copy()
others = others.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
others

,mes,tipo_dia,RatecodeID,personas_transportadas,millas_recorridas,total_servicios
0,2020-01,1,1.0,6952623.0,1.139321e+07,4663589
1,2020-01,1,2.0,185080.0,1.899542e+06,117768
2,2020-01,1,3.0,16205.0,1.673550e+05,10366
3,2020-01,1,4.0,5765.0,7.535857e+04,3864
4,2020-01,1,5.0,37176.0,1.203135e+05,26046
5,2020-01,1,6.0,39.0,2.497000e+01,40
6,2020-01,1,99.0,243.0,3.367000e+02,280
7,2020-01,2,1.0,2323876.0,3.650608e+06,1463042
8,2020-01,2,2.0,63469.0,6.354032e+05,39963
9,2020-01,2,3.0,5116.0,4.882800e+04,3109


In [40]:
others = others[(others['RatecodeID']) != 1 & (others['RatecodeID'] != 2)]
others

,mes,tipo_dia,RatecodeID,personas_transportadas,millas_recorridas,total_servicios
1,2020-01,1,2.0,185080.0,1899541.76,117768
2,2020-01,1,3.0,16205.0,167354.99,10366
3,2020-01,1,4.0,5765.0,75358.57,3864
4,2020-01,1,5.0,37176.0,120313.54,26046
5,2020-01,1,6.0,39.0,24.97,40
6,2020-01,1,99.0,243.0,336.70,280
8,2020-01,2,2.0,63469.0,635403.23,39963
9,2020-01,2,3.0,5116.0,48828.00,3109
10,2020-01,2,4.0,1953.0,25482.00,1266
11,2020-01,2,5.0,15091.0,42658.92,10246


In [41]:
others.drop(columns=['RatecodeID'], inplace=True)
others

/home/david/miniconda3/envs/work/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,mes,tipo_dia,personas_transportadas,millas_recorridas,total_servicios
1,2020-01,1,185080.0,1899541.76,117768
2,2020-01,1,16205.0,167354.99,10366
3,2020-01,1,5765.0,75358.57,3864
4,2020-01,1,37176.0,120313.54,26046
5,2020-01,1,39.0,24.97,40
6,2020-01,1,243.0,336.70,280
8,2020-01,2,63469.0,635403.23,39963
9,2020-01,2,5116.0,48828.00,3109
10,2020-01,2,1953.0,25482.00,1266
11,2020-01,2,15091.0,42658.92,10246


In [42]:
with pd.ExcelWriter('report.xlsx') as writer:  
    jfk.to_excel(writer, sheet_name='JFK')
    regular.to_excel(writer, sheet_name='Regular')
    others.to_excel(writer, sheet_name='Others')

ModuleNotFoundError: No module named 'openpyxl'

# Comprobaciones error de fuente

Fichero CSV original de Enero contiene registros con fechas posteriores y anteriores

In [ ]:
jan = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv')
jan

In [ ]:
jan[(jan['tpep_pickup_datetime'] < '2020-01-01') | (jan['tpep_pickup_datetime'] > '2020-03-31')]